<a href="https://colab.research.google.com/github/shreyasgowdac-319/1BM23CS319-BIS-LAB/blob/main/cuckoo_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math   # use math.gamma and math.sin

# Objective function: Sphere function (minimize)
def objective_function(x):
    # ensure x is a numpy array
    x = np.asarray(x)
    return np.sum(x**2)

# Levy flight step
def levy_flight(Lambda, dim):
    # compute sigma_u using math functions
    sigma_u = (math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2) /
               (math.gamma((1 + Lambda) / 2) * Lambda * 2**((Lambda - 1) / 2)))**(1 / Lambda)
    u = np.random.randn(dim) * sigma_u
    v = np.random.randn(dim)
    step = u / (np.abs(v) ** (1.0 / Lambda))
    return step

def cuckoo_search(n=25, dim=2, max_iter=1000, pa=0.25, lower_bound=-10, upper_bound=10):
    # Initialize nests randomly within bounds
    nests = np.random.uniform(lower_bound, upper_bound, (n, dim))
    fitness = np.array([objective_function(nest) for nest in nests])

    best_idx = np.argmin(fitness)
    best_nest = nests[best_idx].copy()
    best_fitness = fitness[best_idx]

    Lambda = 1.5  # Levy flight parameter
    step_scale = 0.01  # step scaling factor (you can tune this)

    for iteration in range(max_iter):
        for i in range(n):
            # Generate new solution by Levy flight
            step = levy_flight(Lambda, dim)
            step_size = step_scale * step * (nests[i] - best_nest)
            new_solution = nests[i] + step_size
            new_solution = np.clip(new_solution, lower_bound, upper_bound)

            new_fitness = objective_function(new_solution)

            # Replace if better
            if new_fitness < fitness[i]:
                nests[i] = new_solution
                fitness[i] = new_fitness

        # Abandon a fraction pa of worse nests (if num_abandon > 0)
        num_abandon = int(np.round(pa * n))
        if num_abandon > 0:
            worst_indices = np.argsort(fitness)[-num_abandon:]
            nests[worst_indices] = np.random.uniform(lower_bound, upper_bound, (num_abandon, dim))
            for idx in worst_indices:
                fitness[idx] = objective_function(nests[idx])

        # Update best solution
        current_best_idx = np.argmin(fitness)
        current_best_fitness = fitness[current_best_idx]
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_nest = nests[current_best_idx].copy()

        if iteration % 100 == 0 or iteration == max_iter - 1:
            print(f"Iteration {iteration}: Best fitness = {best_fitness:.6f}")

    return best_nest, best_fitness

# Run the algorithm
if __name__ == "__main__":
    best_solution, best_value = cuckoo_search(n=25, dim=2, max_iter=500, pa=0.25)
    print("Best solution found:", best_solution)
    print("Best objective value:", best_value)

Iteration 0: Best fitness = 0.516882
Iteration 100: Best fitness = 0.028269
Iteration 200: Best fitness = 0.002802
Iteration 300: Best fitness = 0.002653
Iteration 400: Best fitness = 0.001122
Iteration 499: Best fitness = 0.000326
Best solution found: [-0.01567098  0.0089473 ]
Best objective value: 0.00032563396557601626
